In [4]:
!pip install requests
!pip install mysql.connector
!pip install pandas
!pip install openpyxl

In [1]:
import mysql.connector
from mysql.connector import Error

# Função para se conectar ao banco de dados e executar a query
def obtain_stiffened_plate_ids():
    try:
        # Conexão com o banco de dados MariaDB
        connection = mysql.connector.connect(
            host='localhost',
            port=3306,
            #database='constructal_automate_results',
            database='constructal_automate_verification_cases',
            user='constructal_automate',
            password='constructal_design_2024'
        )
        
        if connection.is_connected():
            print("Conexão ao banco de dados aberta.")
            cursor = connection.cursor()
            
            # Query SQL
            query = """
                SELECT id FROM csg_stiffenedplate sp
            """
            
            # Executa a query
            cursor.execute(query)
            
            # Busca todos os resultados
            resultados = cursor.fetchall()
            
            # Extrai os ids para a lista stiffened_plates_to_analyse
            stiffened_plates_to_analyse = [row[0] for row in resultados]
            
            print(f"{len(stiffened_plates_to_analyse)} IDs encontrados")
            print(f"{stiffened_plates_to_analyse}")
            return stiffened_plates_to_analyse
            
    except Error as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("Conexão ao banco de dados fechada.")

# Chama a função
stiffened_plates_to_analyse = obtain_stiffened_plate_ids()

Conexão ao banco de dados aberta.
8 IDs encontrados
[1, 2, 3, 4, 5, 6, 7, 8]
Conexão ao banco de dados fechada.


In [2]:
import requests
from requests.auth import HTTPBasicAuth
import time
import pandas as pd

# Configurações da API
STIFFENED_PLATE_ANALYSIS_BASE_URL = 'http://127.0.0.1:8000/stiffened-plate-analysis/'
ELASTIC_BUCKLING_BASE_URL = 'http://127.0.0.1:8000/elastic-buckling/'
ELASTO_PLASTIC_BUCKLING_BASE_URL = 'http://127.0.0.1:8000/elasto-plastic-buckling/'
USERNAME = 'andreiflancanova'
PASSWORD = 'teste1234'

# Função genérica para fazer uma requisição POST e medir o tempo
def make_request(url, request_body, context):
    try:
        start_time = time.time()  # Medir tempo de início

        response = requests.post(url, json=request_body, auth=HTTPBasicAuth(USERNAME, PASSWORD))

        duration = time.time() - start_time  # Duração da requisição

        # Tratamento de erros de resposta HTTP
        if response.status_code == 201:
            response_json = response.json()
            id = response_json.get("id")
            print(f"{context}:")
            print(f"{response_json}")
            return id, duration
        elif response.status_code == 400:
            print(f"{context}: Erro de validação para o payload {request_body}: {response.json()}")
            return None, duration
        elif response.status_code == 401:
            print(f"{context}: Falha na autenticação. Verifique suas credenciais.")
            return None, duration
        else:
            print(f"{context}: Erro inesperado na requisição com payload {request_body}: {response.status_code} - {response.text}")
            return None, duration
    except requests.exceptions.RequestException as e:
        print(f"{context}: Erro de conexão ou de requisição: {e}")
        return None, None

# Funções específicas para cada tipo de requisição
def generate_stiffened_plate_analysis(stiffened_plate_id):
    request_body = {
        "mesh_size": 25,
        "buckling_load_type": 2,
        "case_study": "biaxial-comparacao-lima-2016",
        "material": 1,
        "stiffened_plate": stiffened_plate_id
    }
    return make_request(STIFFENED_PLATE_ANALYSIS_BASE_URL, request_body, "STIFFENED_PLATE_ANALYSIS")

def generate_elastic_buckling(stiffened_plate_analysis_id):
    request_body = {
        "stiffened_plate_analysis": stiffened_plate_analysis_id
    }
    return make_request(ELASTIC_BUCKLING_BASE_URL, request_body, "ELASTIC_BUCKLING")

def generate_elasto_plastic_buckling(stiffened_plate_analysis_id):
    request_body = {
        "stiffened_plate_analysis": stiffened_plate_analysis_id
    }
    return make_request(ELASTO_PLASTIC_BUCKLING_BASE_URL, request_body, "ELASTO_PLASTIC_BUCKLING")

# Função principal para realizar as três requisições em sequência e gerar a planilha
def execute_analises(stiffened_plate_ids):
    dados = []

    for stiffened_plate_id in stiffened_plate_ids:
        # Executar as três requisições e capturar tempos
        id_spa, elapsed_time_spa = generate_stiffened_plate_analysis(stiffened_plate_id)
        if id_spa is None:
            continue  # Se falhar a primeira, pular o resto

        id_eb, elapsed_time_eb = generate_elastic_buckling(id_spa)
        id_epb, elapsed_time_epb = generate_elasto_plastic_buckling(id_spa)

        # Armazenar os tempos no formato desejado
        dados.append({
            'id': id_spa,
            'spa': elapsed_time_spa,
            'eb': elapsed_time_eb,
            'epb': elapsed_time_epb
        })

    # Criar um DataFrame com os dados
    df = pd.DataFrame(dados)

    # Salvar os dados em uma planilha Excel
    df.to_excel('D:/constructal_automate_analysis_files/analyses_duration.xlsx', index=False)

    print("Data saved to sheet.")

# Executar o processamento
execute_analises(stiffened_plates_to_analyse)
#lista_erro = [1904]
#execute_analises(lista_erro)


STIFFENED_PLATE_ANALYSIS:
{'id': 23, 'mesh_size': '25.0', 'num_elem': 4140, 'case_study': 'biaxial-comparacao-lima-2016', 'analysis_dir_path': 'D:/constructal_automate_analysis_files/biaxial-comparacao-lima-2016/F0.1_L2_T2_k6.600_SP1', 'analysis_rst_file_path': None, 'analysis_lgw_file_path': 'D:/constructal_automate_analysis_files/biaxial-comparacao-lima-2016/F0.1_L2_T2_k6.600_SP1/F0.1_L2_T2_k6.600_SP1.txt', 'material': 1, 'stiffened_plate': 1, 'buckling_load_type': 2, 'elastic_buckling_status': 1, 'elasto_plastic_buckling_status': 1}
ELASTIC_BUCKLING:
{'id': 14, 'n_cr': '4240.84', 'sigma_cr_ts': '248.48', 'sigma_cr_ls': '242.27', 'w_center': '0.0007', 'stiffened_plate_analysis': 23}
ELASTO_PLASTIC_BUCKLING:
{'id': 11, 'p_u_ts': '6059.85', 'p_u_ls': '6212.50', 'n_u': '2183.11', 'sigma_u_ts': '127.89', 'sigma_u_ls': '124.75', 'w_max': '3.5446', 'w_dist_img_path': 'D:/constructal_automate_analysis_files/biaxial-comparacao-lima-2016/F0.1_L2_T2_k6.600_SP1/F0.1_L2_T2_k6.600_SP1_w_dist.png'